In [1]:
#create basic components useful for creating a gpkit model

from gpkit import Model, Variable
from gpkit.constraints.tight import Tight
from gpkit.constraints.bounded import Bounded
from gpkit import ureg

​

In [2]:
class Motor(Model):
    
    def setup(self):
        
        constraints = []
        
        #define masses
        self.m_dry = Variable("m_{dry}", "kg", "Dry Mass of Motor")
        self.m_propellant = Variable("m_{prop}", "kg", "Propellant Mass of Motor")
        self.m = Variable("m", "kg", "Total mass of motor")
        
        constraints += [self.m >= self.m_dry + self.m_propellant] 
        
        #define impulse
        self.c = Variable("c", "m/s", "Exhaust speed of motor")
        self.t_burn = Variable("t_{burn}", "s", "Burn Time")
        self.I_t = Variable("I_t", "N s", "Total impulse of motor")
        
        self.F_avg = Variable("T_{avg}", "N", "Average force of motor")
        
        constraints += [Tight([self.F_avg <= self.m_propellant*self.c/self.t_burn])]
        
        constraints += [Tight([self.I_t <= self.m_propellant*self.c])]
        
        return constraints
        
        
class Tube(Model):
    
    def setup(self):
        
        constraints = []
        
        
        self.m = Variable("m", "kg", "Mass of Tube")
        
        self.l = Variable("l", "m", "Length of Tube")
        
        self.C_d = Variable("C_d",  0.2, "", "Drag contribution of Tube")
        
        
        return constraints
        

In [3]:
motor = Motor()

In [4]:
motor

[gpkit.PosynomialInequality(m_Motor >= m_{dry}_Motor + m_{prop}_Motor),
 [gpkit.PosynomialInequality(T_{avg}_Motor <= c_Motor*m_{prop}_Motor*t_{burn}_Motor**-1)],
 [gpkit.PosynomialInequality(I_t_Motor <= c_Motor*m_{prop}_Motor)]]

In [5]:
class Rocket(Model):
    
    def setup(self):
        #basic
        constraints = []
        components = self.components = []
        
        m = self.m  =  Variable("m", "kg","Mass of Rocket")
        
        #motor
        motor = self.motor = Motor()
        components += [motor]
        
        constraints += [motor.m_dry >= 0.2*ureg.kg]
        constraints += [motor.m_propellant >= 0.5*ureg.kg]
        
        tube = self.tube = Tube()
        components += [tube]
        
        constraints += [tube.m >= 0.2*ureg.kg]
        
        constraints += [m >= sum(comp.m for comp in components)]
        return [components, constraints]
        

In [6]:
rocket = Rocket()

In [7]:
rocket

[[[gpkit.PosynomialInequality(m_Rocket/Motor >= m_{dry}_Rocket/Motor + m_{prop}_Rocket/Motor),
   [gpkit.PosynomialInequality(T_{avg}_Rocket/Motor <= c_Rocket/Motor*m_{prop}_Rocket/Motor*t_{burn}_Rocket/Motor**-1)],
   [gpkit.PosynomialInequality(I_t_Rocket/Motor <= c_Rocket/Motor*m_{prop}_Rocket/Motor)]],
  []],
 [gpkit.PosynomialInequality(m_{dry}_Rocket/Motor >= 0.2),
  gpkit.PosynomialInequality(m_{prop}_Rocket/Motor >= 0.5),
  gpkit.PosynomialInequality(m_Rocket/Tube >= 0.2),
  gpkit.PosynomialInequality(m_Rocket >= m_Rocket/Motor + m_Rocket/Tube)]]

In [8]:
optimizer = Model(rocket.m, Bounded(rocket))

In [9]:
optimizer

[[[[[gpkit.PosynomialInequality(m_Rocket/Motor >= m_{dry}_Rocket/Motor + m_{prop}_Rocket/Motor),
     [gpkit.PosynomialInequality(T_{avg}_Rocket/Motor <= c_Rocket/Motor*m_{prop}_Rocket/Motor*t_{burn}_Rocket/Motor**-1)],
     [gpkit.PosynomialInequality(I_t_Rocket/Motor <= c_Rocket/Motor*m_{prop}_Rocket/Motor)]],
    []],
   [gpkit.PosynomialInequality(m_{dry}_Rocket/Motor >= 0.2),
    gpkit.PosynomialInequality(m_{prop}_Rocket/Motor >= 0.5),
    gpkit.PosynomialInequality(m_Rocket/Tube >= 0.2),
    gpkit.PosynomialInequality(m_Rocket >= m_Rocket/Motor + m_Rocket/Tube)]],
  [[gpkit.PosynomialInequality(m_Rocket/Tube <= 1e+30),
    gpkit.PosynomialInequality(m_Rocket/Tube >= 1e-30)],
   [gpkit.PosynomialInequality(m_Rocket/Motor <= 1e+30),
    gpkit.PosynomialInequality(m_Rocket/Motor >= 1e-30)],
   [gpkit.PosynomialInequality(m_Rocket <= 1e+30),
    gpkit.PosynomialInequality(m_Rocket >= 1e-30)],
   [gpkit.PosynomialInequality(c_Rocket/Motor <= 1e+30),
    gpkit.PosynomialInequality(c_R

In [10]:
sol = optimizer.solve()
print(sol.summary())

Using solver 'cvxopt'
Solving for 9 variables.
Solving took 0.0501 seconds.

Cost
----
 0.9 [kg]

Warnings
Unexpectedly Loose Constraints
------------------------------
0.4834 <= 6.985 : T_{avg} <= c*m_{prop}*t_{burn}**-1
0.3073 <= 3.376 : I_t <= c*m_{prop}

Free Variables
--------------
         | Rocket
       m : 0.9     [kg]  Mass of Rocket

         | Rocket/Motor
     I_t : 0.3073  [N·s] Total impulse of motor
 T_{avg} : 0.4834  [N]   Average force of motor
       c : 6.753   [m/s] Exhaust speed of motor
       m : 0.7     [kg]  Total mass of motor
 m_{dry} : 0.2     [kg]  Dry Mass of Motor
m_{prop} : 0.5     [kg]  Propellant Mass of Motor
t_{burn} : 0.4834  [s]   Burn Time

         | Rocket/Tube
       m : 0.2     [kg]  Mass of Tube

Tightest Constraints
--------------------
       | Rocket
    +1 : m_ >= m_/Motor + m_/Tube
 +0.56 : m_{prop}_/Motor >= 0.5
 +0.22 : m_/Tube >= 0.2
 +0.22 : m_{dry}_/Motor >= 0.2

       | Rocket/Motor
 +0.78 : m_ >= m_{dry}_ + m_{prop}_

